In [300]:
import pandas as pd
import numpy as np
import csv

## Demografia

In [272]:
# Verificar Outliers usando Z scores
def abs_z_score(column:pd.Series):
    return ((column-column.mean())/column.std()).abs()
# Verificar Outliers usando o inter-quartile
def is_interquartile(column:pd.Series, percentiles=(25,75))->pd.Series:
    values = column.sort_values().dropna()
    q1,q3 = np.percentile(values, percentiles)
    return (column>q1)&(column<q3)

In [283]:
demography = pd.read_csv("dados/raw/DemographicData_ZCTAs.csv")
demography.drop(columns=["Unnamed: 0"], inplace=True)
demography.dropna(inplace=True)
# filter_0 = (
#     (demography["Population_Under5Years"]==0)&
#     (demography["Population_10to14Years"]==0)&
#     (demography["Population_15to19Years"]==0)&
#     (demography["Population_20to24Years"]==0)&
#     (demography["Population_25to34Years"]==0)&
#     (demography["Population_35to44Years"]==0)&
#     (demography["Population_55to59Years"]==0)&
#     (demography["Population_45to54Years"]==0)
#     )
# # filter_0 = (demography["Population_5to9Years"]==0)&(demography["Population_10to14Years"]==0)
# # demography[demography.columns[demography.columns.str.contains("Population")]]==0
# # demography = demography[~filter_0]
demography = demography[~((demography == 0).astype(int).sum(axis=1)>6)]
demography = demography[abs_z_score(demography["MedianAgeInYears"])<3]
demography = demography[is_interquartile(demography["SexRatio(males per 100 females)"],percentiles=(0.8,99.2))].sort_values("SexRatio(males per 100 females)")

In [284]:
demography.to_csv("dados/processed/DemographicData_ZCTAs.csv", index=False)

## Economia

In [275]:
economic = pd.read_csv("Dados/raw/EconomicData_ZCTAs.csv")
economic.drop("Unnamed: 0", axis=1, inplace=True)
economic.drop_duplicates(inplace=True)

In [276]:
economic.to_csv("dados/processed/EconomicData_ZCTAs.csv", index=False)

## Geocode

In [292]:
geocode = pd.read_csv("dados/raw/df_geocode.csv")
geocode.drop(columns=["Unnamed: 0"], inplace=True)
geocode["Zipcode"]=geocode["Address"].str.extract("(\s([0-9])+$)")[0].str.strip().astype(str)
geocode = geocode[~(geocode["Lab Id"]=="-2")]

In [299]:
geocode.to_csv("dados/processed/LabId_Zipcode.csv", quoting=csv.QUOTE_NONNUMERIC, index=False)

## Test

In [279]:
tests = pd.read_csv("dados/raw/test_data.csv")
tests.drop_duplicates(inplace=True)
tests=tests[~(tests["Testing Cost"]==0)].copy()
tests["Category"]=tests["Category"].str.strip()

In [280]:
tests.to_csv("dados/processed/CodItem_Cost.csv", index=False)

## Transacional

In [281]:
transactional = pd.read_csv("dados/raw/transactional_data.csv",sep=";", decimal=",")
transactional = transactional[~(transactional["Gender"]=="I")]
transactional.drop_duplicates(inplace=True)
transactional = transactional[is_interquartile(transactional["Testing Cost"], percentiles=(0.1,99.9))]
transactional=transactional[~(transactional["Patient Id"]=="9999999-1")]

In [282]:
transactional.to_csv("dados/processed/transactional_data.csv",index=False)